In [9]:
import os
import flwr as fl
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from project import create_model
import tensorflow as tf
import numpy as np

In [10]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(244, 244, 3)),  # Adjust input shape here
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(4, activation="softmax")  # 4 classes
    ])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 242, 242, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 121, 121, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 119, 119, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 59, 59, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 57, 57, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 207936)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │    13,307,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,364,548 (50.98 MB)

 Trainable params: 13,364,548 (50.98 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
train_dir = "../data/Training"
test_dir = "../data/Testing"


def load_data(dir):
    filepaths = []
    labels = []


    folds = os.listdir(dir)

    for fold in folds:
        foldpath = os.path.join(dir, fold)
        
        files = os.listdir(foldpath)
        for f in files:
            fpath = os.path.join(foldpath, f)
            
            filepaths.append(fpath)
            labels.append(fold)

    return pd.DataFrame(data={'filepaths':filepaths, 'labels':labels})

train_data = load_data(train_dir)
test_data = load_data(test_dir)

In [5]:
train_data

,filepaths,labels
0,../data/Training\glioma\Tr-glTr_0000.jpg,glioma
1,../data/Training\glioma\Tr-glTr_0001.jpg,glioma
2,../data/Training\glioma\Tr-glTr_0002.jpg,glioma
3,../data/Training\glioma\Tr-glTr_0003.jpg,glioma
4,../data/Training\glioma\Tr-glTr_0004.jpg,glioma
...,...,...
5707,../data/Training\pituitary\Tr-pi_1452.jpg,pituitary
5708,../data/Training\pituitary\Tr-pi_1453.jpg,pituitary
5709,../data/Training\pituitary\Tr-pi_1454.jpg,pituitary
5710,../data/Training\pituitary\Tr-pi_1455.jpg,pituitary


In [15]:
test_data

,filepaths,labels
0,../data/Testing\glioma\Te-glTr_0000.jpg,glioma
1,../data/Testing\glioma\Te-glTr_0001.jpg,glioma
2,../data/Testing\glioma\Te-glTr_0002.jpg,glioma
3,../data/Testing\glioma\Te-glTr_0003.jpg,glioma
4,../data/Testing\glioma\Te-glTr_0004.jpg,glioma
...,...,...
1306,../data/Testing\pituitary\Te-pi_0295.jpg,pituitary
1307,../data/Testing\pituitary\Te-pi_0296.jpg,pituitary
1308,../data/Testing\pituitary\Te-pi_0297.jpg,pituitary
1309,../data/Testing\pituitary\Te-pi_0298.jpg,pituitary


In [14]:
def load_train_data():
    client_data = load_data(train_dir)
    return client_data.sample(frac=0.5, random_state=np.random.randint(1, 100))
    

In [16]:
load_train_data()

,filepaths,labels
5407,../data/Training\pituitary\Tr-pi_1152.jpg,pituitary
5370,../data/Training\pituitary\Tr-pi_1115.jpg,pituitary
859,../data/Training\glioma\Tr-gl_0859.jpg,glioma
4144,../data/Training\notumor\Tr-no_1484.jpg,notumor
3427,../data/Training\notumor\Tr-no_0767.jpg,notumor
...,...,...
1777,../data/Training\meningioma\Tr-me_0456.jpg,meningioma
3695,../data/Training\notumor\Tr-no_1035.jpg,notumor
5009,../data/Training\pituitary\Tr-pi_0754.jpg,pituitary
2457,../data/Training\meningioma\Tr-me_1136.jpg,meningioma
